In [1]:
# Imports
import plotly.express as px
from helper import *
import pandas as pd
import numpy as np
import gzip
import json
from scipy import stats

## Animated figure

In [2]:
# Load and preprocess pageviews and deaths 
pageview_df = pd.read_csv("page_views_covid_related.csv.gz")
population_df = pd.read_csv("Population_countries.csv")

_,_, df_pageviews100k,_ = get_pageviews_df(pageview_df, population_df, get_country_dict('trust gov mobility'), '2020-01-22', '2020-11-22')
_, _, _, _, deaths100k, _, _,_ = get_cases_deaths_df(population_df, get_country_dict('trust gov mobility'), '2020-01-22', '2020-11-22')
df_pageviews100k = df_pageviews100k.transpose().stack().to_frame().reset_index().rename(columns={"level_0": "country", "date": "date", 0:"pageviews per 100k"}, errors="raise")
deaths100k = deaths100k.transpose().stack().to_frame().reset_index().rename(columns={"level_0": "country", "date": "date", 0:"deaths per 100k"}, errors="raise")
df_animation = df_pageviews100k.merge(deaths100k, on=['country','date'])
df_animation['date'] = pd.to_datetime(df_animation['date'])
df_animation=df_animation.rename(columns={"country": "Country"})

In [3]:
# Dowload and preprocess trust dataset
data_folder = 'data_2/'
df_trust_gov = pd.read_csv(data_folder+'share-who-trust-government.csv.zip') 
df_trust_gov = df_trust_gov.set_index("Entity")[["Trust the national government in this country"]].transpose()[COUNTRY_OWN_LANG_TRUST_GOV.keys()].rename(columns= COUNTRY_OWN_LANG_TRUST_GOV)
country_dict = get_country_dict('trust gov mobility')

# # Map the trust category to the countries in df_animation
# country_dict_cat =  trust_category(df_trust_gov, 5,country_dict)
trust_dict =df_trust_gov.to_dict('index')
df_animation['Trust'] = df_animation['Country'].map(trust_dict['Trust the national government in this country'])
df_animation.head()

,Country,date,pageviews per 100k,deaths per 100k,Trust
0,it,2020-01-22,40.388883,0.0,52.3
1,it,2020-01-23,46.878422,0.0,52.3
2,it,2020-01-24,38.747154,0.0,52.3
3,it,2020-01-25,54.543748,0.0,52.3
4,it,2020-01-26,55.928115,0.0,52.3


In [4]:
data_folder = 'data_2/'

#Download mobility and intervention files
google_mobility = pd.read_csv(data_folder+'Global_Mobility_Report.csv.zip')
interventions = pd.read_csv(data_folder+'interventions.csv.zip')

#Download file with covid pageviews #MODIFY WITH THE NEW DATA
file = gzip.open(data_folder+'aggregated_timeseries.json.gz')
data = json.load(file)

#Change name of the columns
categories = ["Retail and Recreations", "Grocery and Pharmacy", "Parks","Transit stations", "Workplace", "Residential"]
google_mobility = google_mobility.rename(dict(zip(google_mobility.columns[8:],categories)), axis = 1)

#Transform date string to datetime
google_mobility['date'] = pd.to_datetime(google_mobility['date'])

#Delete columns about region
google_mobility = google_mobility.drop(google_mobility.iloc[:,2:7], axis = 1)

#Keep same dates as for corona wikipedia pages
google_mobility = google_mobility.loc[(google_mobility['date'] < "2020-08-01") & (google_mobility['date'] >= "2020-01-22")]

#Regroup all data per country and per date and take the average
mobility = google_mobility.groupby(["country_region_code","country_region","date"]).mean()

#Keep only the country that we are interested in
mobility = mobility[mobility.index.get_level_values('country_region').isin(list(country_dict.keys()))]

# Group together Parks, Retail and Recreations, Transit stations and Workplace by taking the mean of them
mobility['moving category'] = mobility[['Retail and Recreations', 'Parks', 'Transit stations', 'Workplace']].mean(axis=1)

#Group together Grocey and Pharmacy and Residential by taking the mean of them
mobility['covid category'] = mobility[['Grocery and Pharmacy', 'Residential']].mean(axis=1)

moving_cat_df = mobility['moving category'].reset_index()
moving_cat_df['country_region'] =moving_cat_df['country_region'].map(get_country_dict('trust gov mobility'))

# merge dataframe with the mobility dataset
df_animation = df_animation.merge(moving_cat_df, left_on=['Country','date'],  right_on=['country_region','date'])
df_animation = df_animation.drop(columns=['country_region','country_region_code'])
df_animation['date']=df_animation['date'].astype(str)

df_animation['Month-Year'] = pd.to_datetime(df_animation['date']).dt.to_period('M')
df_animation['Month-Year']=df_animation['Month-Year'].astype(str)

inv_map = {v: k for k, v in get_country_dict('trust gov mobility').items()}
df_animation['Country'] = df_animation['Country'].map(inv_map)

/var/folders/yd/qcqcvgvx64j3l0qqnk__2cpm0000gn/T/ipykernel_89658/4206571970.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  google_mobility = pd.read_csv(data_folder+'Global_Mobility_Report.csv.zip')


In [5]:
def f(x):
    d = {}
    d['Cumulative pageviews per 100k'] = x['pageviews per 100k'].sum()
    d['Cumulative deaths per 100k'] = x['deaths per 100k'].sum()
    d["Mobility change from baseline"] = x['moving category'].mean()


    return pd.Series(d, index=['Cumulative pageviews per 100k','Cumulative deaths per 100k', "Mobility change from baseline"])

grouped_df = df_animation.groupby(['Month-Year','Country']).apply(f)
grouped_df = grouped_df.reset_index()
grouped_df =grouped_df.merge(df_animation[['Country','Month-Year','Trust']], how='left', on=['Month-Year','Country']).drop_duplicates()
grouped_df.head(50)

,Month-Year,Country,Cumulative pageviews per 100k,Cumulative deaths per 100k,Mobility change from baseline,Trust
0,2020-02,Bangladesh,0.556806,0.000000,5.250000,83.0
15,2020-02,Bulgaria,161.407208,0.000000,5.097669,40.4
30,2020-02,Croatia,171.629180,0.000000,8.503883,42.0
45,2020-02,Czechia,1925.544347,0.000000,6.410000,45.8
60,2020-02,Denmark,275.594008,0.000000,-1.578199,79.3
75,2020-02,Finland,786.502610,0.000000,5.078715,80.7
90,2020-02,Georgia,57.162566,0.000000,4.208333,57.6
105,2020-02,Germany,1699.497592,0.000000,1.636275,82.0
120,2020-02,Greece,175.065669,0.000000,6.431250,40.7
135,2020-02,Hungary,293.616026,0.000000,9.620652,48.6


In [6]:
# import plotly.io as pio
# pio.templates
# Create animation

def format_title(title, subtitle=None, subtitle_font_size=12):
    title = f'<b>{title}</b>'
    if not subtitle:
        return title
    subtitle = f'<span style="font-size: {subtitle_font_size}px; line-height: 20%;">{subtitle}</span>'
    return f'{title}<br>{subtitle}'

grouped_df["Trust"] = grouped_df["Trust"]/100
df = grouped_df.round(decimals=3)



fig = px.scatter(df,x="Mobility change from baseline", y="Cumulative deaths per 100k" , animation_frame="Month-Year", animation_group="Country",
           size="Cumulative pageviews per 100k" ,color="Trust", hover_name="Country", range_x=[-80,60],range_y=[-20,40], size_max=80, template='plotly_dark' )
      
fig.update_layout(height=500, width= 900,title_text="Cumulative deaths per 100k vs. Mobility change from baseline, 2020")
fig.update_layout(margin=dict(l=40, r=10, t=125, b=10))  
fig.add_annotation(xref='paper',yref='paper',x=0, y=1.28,showarrow=False,text = "The x-axis shows the mean mobility change from baseline per month.")
fig.add_annotation(xref='paper',yref='paper',x=0, y=1.22,showarrow=False,text = "The y-axis shows the cumulative deaths per 100k per month.")
fig.add_annotation(xref='paper',yref='paper',x=0, y=1.16,showarrow=False,text = "The size of the bubble shows the cumulative pageviews per 100k per month ")
fig.add_annotation(xref='paper',yref='paper',x=0, y=1.10,showarrow=False,text = "The colour of the bubble corresponds to how much people trust their government.")
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 2000
fig.show()

fig.write_html('/Users/charlottsertic/Documents/GitHub/ada-2022-project-thedatadiggers22/data_2/Images/bubble_plot.html')


In [7]:
# Calculate the mean of the mobilitiy baseline for every country 



## TRUSTING GOVERNMENT AND MOBILITY

In [8]:

mobility_trust = mobility['moving category'].reset_index()
mobility_trust['country_region'] =mobility_trust['country_region'].map(get_country_dict('trust gov mobility'))
mobility_trust = mobility_trust.groupby('country_region')['moving category'].min().to_frame()
mobility_trust = mobility_trust.reset_index()
mobility_trust['Trust the national government in this country'] = mobility_trust['country_region'].map(trust_dict['Trust the national government in this country'])
mobility_trust['country_region'] = mobility_trust['country_region'].map(inv_map)
mobility_trust = mobility_trust.rename(columns={"moving category": "Min mobility change from baseline","country_region": "Country"})
mobility_trust.head()



,Country,Min mobility change from baseline,Trust the national government in this country
0,Bulgaria,-62.327553,40.4
1,Bangladesh,-61.750000,83.0
2,Czechia,-65.633333,45.8
3,Denmark,-35.839475,79.3
4,Germany,-56.426471,82.0


In [9]:
fig = px.scatter(mobility_trust, x='Trust the national government in this country', y="Min mobility change from baseline",hover_name="Country",trendline="ols", title='Relationship between min moblitiy change from baseline and trust in the national government in a country, 2020')

# fig.update_layout(height=800,title_text='Relationship between min moblitiy change from baseline and trust in the national government in a country')

fig.update_layout(margin=dict(l=20, r=20, t=100, b=0))  
fig.add_annotation(xref='paper',yref='paper',x=0, y=1.13,showarrow=False,text = "The x-axis the trust in the national government in a country.")
fig.add_annotation(xref='paper',yref='paper',x=0, y=1.09,showarrow=False,text = "The y-axis shows the min mobility change from baseline per month.")
fig.update_layout(yaxis_ticksuffix = "%")
fig.update_layout(xaxis_ticksuffix = "%")
fig.show()

# mobility change from baseline
corr, p_value = stats.pearsonr(mobility_trust['Trust the national government in this country'], mobility_trust["Min mobility change from baseline"])
print("Correlation coefficient: {}".format(corr))
print("P-value: {}".format(p_value))

Correlation coefficient: 0.4763412530416577
P-value: 0.007790561016220846
